## Simple Time Series Prediction model with Prophet
### Scott Hendrickson


In [ ]:
import pandas as pd

from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation

from matplotlib import pyplot as plt
%matplotlib inline

## Canonical Example for Prophet Website

In [ ]:
!ls ../data
df = pd.read_csv('../data/example_wp_log_peyton_manning.csv')
df.head()

### Many options, but a basic prophet model on daily data does the obvious seasonality detection

In [ ]:
m = Prophet()
m.fit(df)

### Even has a utility to create a data frame with historical and future predictions

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast, figsize=[15,10])

In [ ]:
fig2 = m.plot_components(forecast, figsize=[15,15])

## Get an idea of prediction quality using cross-validation

In [ ]:
df_cv = cross_validation(m, initial='730 days', period='365 days', horizon = '365 days')

In [ ]:
df_p = performance_metrics(df_cv)
print(len(df))
df_p.head(10)

# Deploy and Use Model

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
#base_url = "http://localhost:5000"
base_url = "http://192.168.127.8/ts-model"
#base_url = "http://127.0.0.1"

In [ ]:
res = requests.get(base_url + "/version")
print(res.json())

In [ ]:
res = requests.get(base_url + "/example")
print(res.json())

In [ ]:
dft = pd.DataFrame(res.json()["data"], columns=res.json()["header"])
dft["ds"] = pd.to_datetime(dft['ds'])
dft["y"] = dft['y'].astype(float)
dft.plot("ds", "y", figsize=[15,10])

In [ ]:
tr = requests.post(base_url + "/train", json = res.json())
print(tr)
print(tr.json())
model_id = tr.json()["model_id"]

In [ ]:
print(model_id)
xv = requests.get(base_url + f"/validation/{model_id}", timeout=120)
print(xv)
print(xv.json())

In [ ]:
pr = requests.post(base_url + f"/predict", json = {"size": 180, "data":[], "model_id": model_id})

In [ ]:
print(pr.json()["header"])
dfr = pd.DataFrame(pr.json()["data"], columns=pr.json()["header"])
dfr["ds"] = pd.to_datetime(dfr['ds'])
ax = dfr.plot("ds",["yhat", "yhat_upper", "yhat_lower"], figsize=[12,12], alpha=0.6)
dft.plot("ds", "y", ax=ax, style="y*", alpha=0.4)